In [1]:
library(dplyr)
library(rstatix)
library(ggplot2)
library(pals)
library(cowplot)
library(Seurat)
library(SCP)
library(scCustomize) 
library(SCpubr)
library(ggsci)
library(ggpubr)
library(biomaRt)
library(muscat)
source('helper_functions.R')

set.seed(123)

root.dir <- "/project/bicistronic_carT_gbm_Jackie/"
figures.dir <- paste0(root.dir, "Final/", "Figures/", "myeloid/") # figures will be outputted in this folder
dir.create(file.path(figures.dir, "supp")) # supplemental figures will be outputted in this subfolder


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘rstatix’


The following object is masked from ‘package:stats’:

    filter


Loading required package: SeuratObject

Loading required package: sp


Attaching package: ‘SeuratObject’


The following objects are masked from ‘package:base’:

    intersect, t




Conda not found.

If you have already created an SCP python environment using conda, you can specify the conda path by setting options(reticulate.conda_binary = "/path/to/conda", SCP_env_name = "SCP_env") before loading the package.

scCustomize v3.0.1
If you find the scCustomize useful please cite.
See 'samuel-marsh.github.io/scCustomize/articles/FAQ.html' for citation info.

---------------------------------------------------------------

SCpubr


If you use SCpubr in your research, please cite it accordingly:

In [2]:
# correct ordering for figures
myeloid.cell.types <- c('Pro-inflammatory macrophage', 'Inhibitory macrophage', 'CX3CR1+ monocyte', 'Monocyte', 'cDC1',  'pDC', 'cDC2', 'mregDC')


In [3]:
# for plotting
col.pal <- stepped3(20)
all.patients <- c("P6", "P1", "P8", "P5", "P2", "P7", "P4", "P3")
col.pal.values <- c(col.pal[5], col.pal[6], col.pal[7], col.pal[8], col.pal[11], col.pal[3], col.pal[2], col.pal[1])
scale.color.patient <- scale_color_manual(breaks = all.patients, values = col.pal.values)
scale.fill.patient <- scale_fill_manual(breaks = all.patients, values = col.pal.values)

# Start here with myeloid seurat object

In [4]:
myeloid <- readRDS(paste0(root.dir, "ProcessedData/seurat/annotated_myeloid_refined.rds"))
myeloid$cell_type <- factor(myeloid$cell_type, levels = myeloid.cell.types)
myeloid$Patient <- factor(myeloid$Patient, levels = all.patients)
myeloid$response.status <- sapply(myeloid$Patient, function(x) ifelse(x %in% c("P3", "P4", "P7", "P2"), "Responder", "Non-responder"))
myeloid$response.status <- factor(myeloid$response.status, levels = c("Non-responder", "Responder"))

In [5]:
myeloid$cell_type_broad <- sapply(myeloid$cell_type, function(x) 
    case_when(
        x %in% c('Pro-inflammatory macrophage', 'CX3CR1+ monocyte', 'Inhibitory macrophage', 'Monocyte') ~ 'Mono/Mac',
        TRUE ~ x
    )
)

In [6]:
# cell type label with counts for each cell type
cell_counts <- table(myeloid$cell_type)
new_labels <- paste0(names(cell_counts), " (", cell_counts, ")")
names(new_labels) <- names(cell_counts)

myeloid$`Cell type` <- plyr::mapvalues(
  x = myeloid$cell_type,
  from = names(new_labels),
  to = new_labels
)

myeloid$`Cell type` <- factor(myeloid$`Cell type`, levels = unname(new_labels[myeloid.cell.types]))

In [7]:
# Figure 5A
pdf(paste0(figures.dir, "5A myeloid UMAP.pdf"), width = 8, height = 6)
CellDimPlot(srt = myeloid, group.by = c("Cell type"), show_stat=F, palette='Dark2',
    label=F, label_insitu=F, label.fg = "black", label.bg = "white", label.bg.r = 0, label.size =4, label_repel = F, pt.size=0.1,
    ncol=1, reduction = "umap.harmony") & xlab('UMAP 1')  & ylab('UMAP 2') 
dev.off()

Warning message:
“No shared levels found between `names(values)` of the manual scale and the
data's fill values.”


pdf 
  2

In [8]:
# Supplemental Figure 6D
genes <- c("SPP1", "CXCL9")
pdf(paste0(figures.dir, "supp/", "Supp 6D SPP1 CXCL9 myeloid UMAP.pdf"), width = 16, height = 6)
FeaturePlot(myeloid, features = genes, cols = c("grey", "red"), order=T, reduction = "umap.harmony", ncol=length(genes),
            min.cutoff='q30', max.cutoff='q90', label = F, pt.size=0.1) & xlab('UMAP 1') & ylab('UMAP 2') & theme(
        axis.text=element_blank(),axis.line = element_blank(), 
        axis.ticks=element_blank())
dev.off()

Warning message:
“The `slot` argument of `FetchData()` is deprecated as of SeuratObject 5.0.0.
ℹ Please use the `layer` argument instead.
ℹ The deprecated feature was likely used in the Seurat package.
  Please report the issue at <https://github.com/satijalab/seurat/issues>.”
Warning message:
“The following variables were found in both object meta data and the default assay: CXCL9
Returning meta data; if you want the feature, please use the assay's key (eg. rna_CXCL9)”


pdf 
  2

In [9]:
# Figure 5E
pdf(paste0(figures.dir, "5E myeloid proportions by day.pdf"), width = 6, height = 6)
CellStatPlot(myeloid, stat.by = "cell_type", group.by = "Day", label = F, alpha=1, palette='Dark2')
dev.off()

pdf 
  2

In [10]:
# Supplemental Figure 6A
d0 <- CellStatPlot(subset(myeloid, subset = Day == "D0"), stat.by = "cell_type", group.by = "response.status", label = F, alpha=1, palette='Dark2') + ggtitle("Day 0")
d7 <- CellStatPlot(subset(myeloid, subset = Day == "D7"), stat.by = "cell_type", group.by = "response.status", label = F, alpha=1, palette='Dark2') + ggtitle("Day 7")
d21 <- CellStatPlot(subset(myeloid, subset = Day == "D21"), stat.by = "cell_type", group.by = "response.status", label = F, alpha=1, palette='Dark2') + ggtitle("Day 21")
pdf(paste0(figures.dir, "supp/", "Supp 6A myeloid proportions by response by day.pdf"), width = 16, height = 6)
p <- d0 + d7 + d21
print(p)
dev.off()

pdf 
  2

In [11]:
# Supplemental Figure 6B
d0 <- CellStatPlot(subset(myeloid, subset = Day == "D0"), stat.by = "cell_type", group.by = "Patient", label = F, alpha=1, palette='Dark2') + ggtitle("Day 0")
d7 <- CellStatPlot(subset(myeloid, subset = Day == "D7"), stat.by = "cell_type", group.by = "Patient", label = F, alpha=1, palette='Dark2') + ggtitle("Day 7")
d21 <- CellStatPlot(subset(myeloid, subset = Day == "D21"), stat.by = "cell_type", group.by = "Patient", label = F, alpha=1, palette='Dark2') + ggtitle("Day 21")
pdf(paste0(figures.dir, "supp/", "Supp 6B myeloid proportions by Patient by day.pdf"), width = 24, height = 6)
p <- d0 + d7 + d21
print(p)
dev.off()

pdf 
  2

In [12]:
# Figure 5B
genes <- c(
    'IL1B', 'TNF', 'IL6', 
    "SPP1", "C1QA", "C1QB", "C1QC", 
    'APOE','TREM2', 'FOLR2','MS4A4A', 
    "CX3CR1", 
    'VCAN','FCN1','CD300E','LILRA5',
    'XCR1','CLEC9A','CADM1','THBD',
    'LILRA4', 'LRRC26','IL3RA','CLEC4C',
    'CD1C','CD1E','FCER1A','CD5',
    'LAMP3','CCR7','CCL19','BIRC3'
) 

myeloid$cell_type <- factor(myeloid$cell_type, levels = rev(myeloid.cell.types))
Idents(myeloid) <- "cell_type"
pdf(paste0(figures.dir, "5B myeloid subtypes gene signatures dotplot.pdf"), width = 12, height = 4)
p <- Clustered_DotPlot(myeloid,  features = genes, k = 1, x_lab_rotate=90,
        cluster_feature = F, cluster_ident = F, flip=T, exp_color_max=1, exp_color_min=-1,  
        show_ident_colors=F,  colors_use_exp = colorRampPalette(c("darkgreen","white","darkorchid4"))(50)) 
dev.off()

Warning message:
“The following variables were found in both object meta data and the default assay: IL1B, TNF, IL6, APOE, TREM2, FOLR2, MS4A4A, VCAN, FCN1, CD300E, LILRA5, XCR1, CLEC9A, CADM1, THBD, LILRA4, LRRC26, IL3RA, CLEC4C, CD1C, CD1E, FCER1A, CD5, LAMP3, CCR7, CCL19, BIRC3
Returning meta data; if you want the feature, please use the assay's key (eg. rna_IL1B)”


pdf 
  2

# DEG analysis

In [13]:
muscat.edgeR <- muscat.de.genes(
    seurat_obj = myeloid, 
    cluster_col = 'cell_type_broad', 
    group_col = 'Day', 
    sample_col = 'sample_id', 
    method = 'edgeR',
    filter = "both"
)

Warning message:
“`PackageCheck()` was deprecated in SeuratObject 5.0.0.
ℹ Please use `rlang::check_installed()` instead.
ℹ The deprecated feature was likely used in the Seurat package.
  Please report the issue at <https://github.com/satijalab/seurat/issues>.”
Warning message:
“The `slot` argument of `GetAssayData()` is deprecated as of SeuratObject 5.0.0.
ℹ Please use the `layer` argument instead.
ℹ The deprecated feature was likely used in the Seurat package.
  Please report the issue at <https://github.com/satijalab/seurat/issues>.”


  |======================================================================| 100%

         Num DE.genes % DE.genes
cDC1              202      0.552
cDC2              196      0.535
Mono/Mac          609      1.664


In [14]:
protein_coding <- filter_genes_by_biotype(muscat.edgeR$gene, biotype = "protein_coding")
muscat.edgeR <- muscat.edgeR[muscat.edgeR$gene %in% c(protein_coding$filtered_genes, 'CAR'),]

In [15]:
ctype <- "Mono/Mac"

muscat.edgeR.ctype <- muscat.edgeR %>% filter(cluster_id == ctype)

muscat.edgeR.up <- muscat.edgeR.ctype %>% filter(p_adj.loc <= 0.05, logFC > 1) %>% arrange(p_adj.loc)
genes.up <- unique(muscat.edgeR.up$gene)

muscat.edgeR.down <- muscat.edgeR.ctype %>% filter(p_adj.loc <= 0.05, logFC < -1) %>% arrange(p_adj.loc)
genes.down <- unique(muscat.edgeR.down$gene)

In [16]:
# Figure 5C
muscat.edgeR.ctype <- muscat.edgeR %>% 
    filter(cluster_id == ctype) %>% 
    dplyr::select(gene, p_adj.loc, logFC) 

rownames(muscat.edgeR.ctype) <- muscat.edgeR.ctype$gene
colnames(muscat.edgeR.ctype) <- c('gene', 'p_val_adj', 'avg_log2FC')

muscat.edgeR.ctype <- muscat.edgeR.ctype %>% dplyr::select(p_val_adj, avg_log2FC)

pdf(file.path(figures.dir, paste0("5C MonoMac D21 vs D0 log2fc.pdf")), width=8, height=8)
do_VolcanoPlot(sample = myeloid, genes.up = genes.up[1:10], genes.down = genes.down[1:10],
            de_genes = muscat.edgeR.ctype, pt.size=0.6, min.segment.length=0.01, force=20, nudge_x=1.5,
            pval_cutoff = 0.05,
            FC_cutoff = 1, n_genes = 10, order_tags_by='custom') +
            xlab("log2 fold change (Day 21 vs Day 0)") +
            ggtitle(ctype)
dev.off()

Warning message:
“Removed 1 row containing missing values or values outside the scale range
(`geom_text_repel()`).”


pdf 
  2

In [17]:
# Supplemental Figure 6C
deg.ctypes <- unique(muscat.edgeR$cluster_id)
deg.ctypes <- deg.ctypes[deg.ctypes != "Mono/Mac"]

plot.list <- list()
for (ctype in deg.ctypes) {
    print(ctype)
    
    muscat.edgeR.ctype <- muscat.edgeR %>% filter(cluster_id == ctype)
    
    muscat.edgeR.up <- muscat.edgeR.ctype %>% filter(p_adj.loc <= 0.05, logFC > 1) %>% arrange(p_adj.loc)
    genes.up <- unique(muscat.edgeR.up$gene)
    
    muscat.edgeR.down <- muscat.edgeR.ctype %>% filter(p_adj.loc <= 0.05, logFC < -1) %>% arrange(p_adj.loc)
    genes.down <- unique(muscat.edgeR.down$gene)

    muscat.edgeR.ctype <- muscat.edgeR %>% 
    filter(cluster_id == ctype) %>% 
    dplyr::select(gene, p_adj.loc, logFC) 

    rownames(muscat.edgeR.ctype) <- muscat.edgeR.ctype$gene
    colnames(muscat.edgeR.ctype) <- c('gene', 'p_val_adj', 'avg_log2FC')
    
    muscat.edgeR.ctype <- muscat.edgeR.ctype %>% dplyr::select(p_val_adj, avg_log2FC)
    
    p <- do_VolcanoPlot(sample = myeloid, genes.up = genes.up[1:10], genes.down = genes.down[1:10],
                de_genes = muscat.edgeR.ctype, pt.size=0.6, min.segment.length=0.01, force=20, nudge_x=1.5,
                pval_cutoff = 0.05,
                FC_cutoff = 1, n_genes = 10, order_tags_by='custom') +
                xlab("log2 fold change (Day 21 vs Day 0)") +
                ggtitle(ctype) 

    plot.list[[ctype]] <- p
}
p <- plot_grid(plotlist = plot.list)
pdf(file.path(figures.dir, "supp", paste0("Supp 6C DC D21 vs D0 log2fc.pdf")), width=16, height=8)
print(p)
dev.off()

[1] "cDC1"
[1] "cDC2"


Warning message:
“Removed 1 row containing missing values or values outside the scale range
(`geom_text_repel()`).”


pdf 
  2

# Myeloid programs
From the following paper:  
Miller, T. E. et al. Programs, origins and immunomodulatory functions of myeloid cells in glioma. Nature 640, 1072–1082 (2025).

In [18]:
miller.programs <- read.csv(paste0(root.dir, "Resources/", "Tyler_Myeloid_Programs.csv"))

In [19]:
# function for adding module score and then correcting the metadata column names (removes extra digits)
add.module.score <- function(seurat.obj, gene.signatures) {

  # remove existing  
  for (g in names(gene.signatures)) {
    if (g %in% colnames(seurat.obj@meta.data)) {
        seurat.obj@meta.data[[g]] <- NULL
    }
  }
    
  seurat.obj <- AddModuleScore(
    object = seurat.obj,
    features = gene.signatures,
    name = names(gene.signatures),
    search = T
  )
  
  i <- 1
  for (g in names(gene.signatures)) {
    colnames(seurat.obj@meta.data) <- gsub(paste0(g, i), g, colnames(seurat.obj@meta.data))
    i <- i + 1
  }
  
  return(seurat.obj)
}

In [20]:
# add gene signature
gene.signatures <- list()

for (col in colnames(miller.programs)) {
    gene.signatures[[col]] <- miller.programs[[col]]
}

myeloid <- add.module.score(myeloid, gene.signatures)

In [21]:
signatures <- c(
    'Complement.Immunosuppressive',
    'Scavenger.Immunosuppressive',
    'Microglial.Inflammatory',
    'Systemic.Inflammatory',
    'Hypoxia',
    'IFN.Response',
    'HS.UPR','G2.M','G1.S'
) 

In [22]:
df <- myeloid@meta.data %>% 
    dplyr::select(Patient, response.status, Day, sample_id, cell_type_broad_l1, names(gene.signatures)) %>%
    pivot_longer(names(gene.signatures), names_to = "program", values_to = "expression") %>%
    group_by(Patient, response.status, Day, sample_id, cell_type_broad_l1, program) %>%
    summarize(mean.expression = mean(expression), .groups="drop")

In [23]:
# Figure 5D
my_comparisons <- list(c("D0", "D7"), c("D0", "D21"))
program.of.interest <- "Complement.Immunosuppressive"

plot.list <- list()
for (celltype in c("Mono/Mac")) {
    program.df <- df %>% filter(program == program.of.interest, cell_type_broad_l1 == celltype) %>% arrange(Patient, Day)

    stat.test <- program.df %>%
        group_by(cell_type_broad_l1) %>%
        wilcox_test(mean.expression ~ Day, paired = T, comparisons = my_comparisons) %>%
        adjust_pvalue(method = "fdr") %>%
        add_significance("p.adj") %>%
        add_xy_position(x = "Day")
    stat.test$p.adj.label <- apply(stat.test, 1, function(row) ifelse(as.numeric(row[["p.adj"]]) <= 0.1, 
                                                                      ifelse(as.numeric(row[["p.adj"]]) <= 0.05, row[["p.adj.signif"]], row[["p.adj"]]), 
                                                                      NA))
                                    
    p <- ggplot(program.df, aes(Day, mean.expression)) +
        geom_boxplot(outlier.shape = NA) + 
        geom_point(position=position_jitter(width=0.2)) +   
        theme_bw() +
        ylab(paste0(program.of.interest, " signature")) +
        ggtitle(celltype) +
        stat_pvalue_manual(stat.test, label = "p.adj")
    plot.list[[celltype]] <- p
}
p <- plot_grid(plotlist = plot.list)
pdf(paste0(figures.dir, "5D ", program.of.interest, " Miller signature by day.pdf"), width = 6, height = 6)
print(p)
dev.off()

pdf 
  2

In [24]:
sessionInfo() 

R version 4.3.2 (2023-10-31)
Platform: aarch64-unknown-linux-gnu (64-bit)
Running under: Ubuntu 22.04.3 LTS

Matrix products: default
BLAS:   /usr/lib/aarch64-linux-gnu/openblas-pthread/libblas.so.3 
LAPACK: /usr/lib/aarch64-linux-gnu/openblas-pthread/libopenblasp-r0.3.20.so;  LAPACK version 3.10.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Etc/UTC
tzcode source: system (glibc)

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] purrr_1.0.4        tidyr_1.3.1        muscat_1.16.0      biomaRt_2.58.2    
 [5] ggpubr_0.6.0       ggsci_3.2.0        SCpubr_1.1.1.9000  scCustomize_3.0.1 
